In [11]:
import ray

In [9]:
@ray.remote
def run_exec():
    import sys
    sys.path.append('../')
    from fl_strategies.fed_avg import FedAvg
    import yaml
    from experiments import dlrm 
    from fl_strategies import fed_avg
    from fedrec.utilities.logger import NoOpLogger
    from fedrec.python_executors.aggregator import AggregatorConfig

    from fedrec.multiprocessing.process_manager import RayProcessManager
    from executor import JobExecutor
    with open("../configs/dlrm_fl.yml", 'r') as cfg:
        config = yaml.load(cfg, Loader=yaml.FullLoader)

    ag_config = {
            # Seed for RNG used in shuffling the training data.
        "data_seed" : 100,
        # Seed for RNG used in initializing the model.
        "init_seed" : 100,
        # Seed for RNG used in computing the model's training loss.
        # Only relevant with internal randomness in the model, e.g. with dropout.
        "model_seed" : 100
    }
    aggregator_cfg = AggregatorConfig(**ag_config)
    JobExecutor = ray.remote(JobExecutor)
    je_Aggregator = JobExecutor.remote(FedAvg,
                                    config,
                                    aggregator_cfg,
                                    NoOpLogger())
    je_Aggregator.run.remote()

run_exec.remote()

ObjectRef(ee4e90da584ab0ebffffffffffffffffffffffff0100000001000000)

2021-11-14 22:05:44,164	ERROR worker.py:79 -- Unhandled error (suppress with RAY_IGNORE_UNHANDLED_ERRORS=1): ray::run_exec() (pid=106524, ip=10.160.0.3)
  File "/tmp/ipykernel_105374/3069150239.py", line 27, in run_exec
  File "/home/varun/RecoEdge/notebooks/../fedrec/multiprocessing/process_manager.py", line 52, in __init__
    ray.init()
RuntimeError: Maybe you called ray.init twice by accident? This error can be suppressed by passing in 'ignore_reinit_error=True' or by calling 'ray.shutdown()' prior to 'ray.init()'.


In [10]:
ray.shutdown()

In [ ]:
rpm = RayProcessManager()
rpm.distribute(JobExecutor, FedAvg.__name__, 1 ,
                        FedAvg, config["model"], aggregator_cfg, NoOpLogger())

rpm.start(FedAvg.__name__, "run")